In [1]:
import pandas as pd
import numpy as np
import os
import tarfile

In [2]:
data_path='J:/VeReMi-Dataset/000_Select_data'
os.chdir(data_path)
datalist=os.listdir(data_path)

In [4]:
df_list=[]
t=0
label=[]
for i in datalist:
    tar = tarfile.open(i, "r:gz")
    print(i)
    for member in tar.getmembers():
         f = tar.extractfile(member)
         if member.name.find("Truth")==-1:
             if f is not None:
                 label.append(int(member.name.split('/')[1].split('-')[3][1:]))
                 t=pd.read_json(f.read(),lines=True)
                 df_list.append(t)

VeReMi_54000_57600_2019-11-28_10-42-31.tar.gz


In [5]:
import numpy as np
import gc
normal=[]
dos=[]
dataset=np.vstack((df_list,label))
#del df_list
#gc.collect(df_list)
'''loc=0
for i in dataset[1]:
    loc+=1
    if i == 0:
        normal.append(dataset[0][loc])
    elif i == 1:
        dos.append(dataset[0][loc])'''



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


'loc=0\nfor i in dataset[1]:\n    loc+=1\n    if i == 0:\n        normal.append(dataset[0][loc])\n    elif i == 1:\n        dos.append(dataset[0][loc])'

In [6]:
ratio_train = 0.8 #训练集比例
ratio_val = 0.1 #验证集比例
ratio_test = 0.1 #测试集比例
assert (ratio_train + ratio_val + ratio_val) == 1.0,'Total ratio Not equal to 1' ##检查总比例是否等于1
cnt_test = round(dataset.shape[1] * ratio_test ,0)
cnt_val = round(dataset.shape[1] * ratio_val ,0)
cnt_train = dataset.shape[1] - cnt_test - cnt_val
print("test Sample:" + str(cnt_test))
print("val Sample:" + str(cnt_val))
print("train Sample:" + str(cnt_train))

test Sample:68.0
val Sample:68.0
train Sample:545.0


In [7]:
np.random.shuffle(dataset) ##打乱列表
train_list=[[]]
val_list=[]
test_list=[]
t=dataset[:,1]
for i in range(int(cnt_train)):
    #np.append(train_list,dataset[:,i],axis=1)
    train_list.append(dataset[:,i])
for i in range(int(cnt_train) ,int(cnt_train + cnt_val)):
    val_list.append(dataset[:,i])

for i in range(int(cnt_train + cnt_val) ,int(cnt_train + cnt_val + cnt_test)):
    test_list.append(dataset[:,i])
print(1)
#del dataset
#gc.collect(dataset)
del(train_list[0])
del(val_list[0])
del(test_list[0])

1


In [8]:
np.save('train.npy',train_list)
np.save('val.npy',val_list)
np.save('test.npy',test_list)